In [12]:
import numpy as np

DATA_PATH = "/public/liujiawei/huawei/ZHB/ADF-master/mnist/"
X_train = np.load(DATA_PATH+"mnist_train.npy")
Y_train = np.load(DATA_PATH+"mnist_train_label.npy")
X_test = np.load(DATA_PATH+"mnist_test.npy")
Y_test = np.load(DATA_PATH+"mnist_test_label.npy")

In [13]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [14]:
input_shape = 28*28

model = Sequential()
model.add(Dense(input_shape=(input_shape,), units=input_shape, activation='relu'))
# model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=10, activation='softmax'))
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train,epochs=1,batch_size=256)

MODEL_PATH = "/public/liujiawei/huawei/ZHB/ADF-master/models/"
model.save(MODEL_PATH+"mnist_FC4_bad.h5")

del model

Train on 60000 samples
60000/60000 [==============================] - 1s 16us/sample - loss: 1.4401 - acc: 0.6664


In [15]:
MODEL_PATH = "/public/liujiawei/huawei/ZHB/ADF-master/models/"
model = load_model(MODEL_PATH+"mnist_FC4_bad.h5")

In [16]:
loss_acc = model.evaluate(X_test, Y_test, batch_size=128)
print('accuracy', loss_acc[-1])

10000/10000 [==============================] - 0s 19us/sample - loss: 0.7901 - acc: 0.8420
accuracy 0.842


In [17]:
Gini = []
index1 = 0
index2 = 0
for x_tmp in X_test:
    per_tmp = model.predict(x_tmp.reshape([-1,28*28]))
    label_tmp = np.argmax(per_tmp)
    if label_tmp == np.argmax(Y_test[index1]):
        index2 = index2 +1
    index1 = index1 + 1
    Gini_tmp = 1-np.sum(per_tmp*per_tmp)
    Gini.append(Gini_tmp)

In [18]:
print('test number:', index1, '\t misclassified number:', index1-index2)

test number: 10000 	 misclassified number: 1580


In [19]:
indexs = np.argsort(Gini)
indexs = indexs[::-1]

In [20]:
o_i = 0
for i in range(0, (index1-index2)):
    x_tmp = X_test[indexs[i]].reshape([-1,28*28])
    y_tmp = Y_test[indexs[i]]
    per_tmp = model.predict(x_tmp)
    if np.argmax(y_tmp)!=np.argmax(per_tmp):
        o_i = o_i+i
APFD = 1-o_i/(index1*(index1-index2))+1/(2*index1)
print(APFD)

0.9658267721518987


In [31]:
def softmax( f ):
    # instead: first shift the values of f so that the highest number is 0:
    f -= np.max(f) # f becomes [-666, -333, 0]
    return np.exp(f) / np.sum(np.exp(f))  # safe to do, gives the correct answer
    
from tensorflow.keras import backend as BE
import tensorflow as tf

get_activations = BE.function(inputs=model.inputs[0], outputs=model.layers[-3].output[:,:])

Gini_act = []
for x_tmp in X_test:
    x_act = get_activations(x_tmp.reshape([-1,28*28]))
    Gini_tmp = 1-np.sum(x_act*x_act)
    Gini_act.append(Gini_tmp)


In [29]:
indexs = np.argsort(Gini_act)
indexs = indexs[::-1]
o_i = 0
for i in range(0, (index1-index2)):
    x_tmp = X_test[indexs[i]].reshape([-1,28*28])
    y_tmp = Y_test[indexs[i]]
    per_tmp = model.predict(x_tmp)
    if np.argmax(y_tmp)!=np.argmax(per_tmp):
        o_i = o_i+i
APFD = 1-o_i/(index1*(index1-index2))+1/(2*index1)
print(APFD)

0.9858898101265823


In [ ]:
0.9832627215189873 倒数第一层
0.9858898101265823 倒数第二层